In [1]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoProcessor, AutoTokenizer
import time
import numpy as np
import cv2
import textwrap
from PIL import Image, ImageDraw, ImageFont
import enum
import json
import os 


/home/zhekai/miniforge3/envs/vllm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VLLM

In [2]:
from vllm import LLM, SamplingParams
language_model = LLM("logs/llama-bridge",  trust_remote_code=True, gpu_memory_utilization=0.8, device="cuda:0")


INFO 01-24 15:12:00 __init__.py:183] Automatically detected platform cuda.


2025-01-24 15:12:00,385	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 01-24 15:12:04 config.py:528] This model supports multiple tasks: {'score', 'reward', 'classify', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 01-24 15:12:04 llm_engine.py:232] Initializing an LLM engine (v0.1.dev4262+g6609cdf) with config: model='logs/llama-bridge', speculative_config=None, tokenizer='logs/llama-bridge', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda:0, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=logs/ll

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:00,  2.51it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:00<00:00,  2.00it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:01<00:00,  1.86it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:01<00:00,  1.93it/s]


INFO 01-24 15:12:06 model_runner.py:1115] Loading model weights took 12.5527 GB


INFO 01-24 15:12:07 worker.py:266] Memory profiling takes 0.76 seconds
INFO 01-24 15:12:07 worker.py:266] the current vLLM instance can use total_gpu_memory (23.64GiB) x gpu_memory_utilization (0.80) = 18.91GiB
INFO 01-24 15:12:07 worker.py:266] model weights take 12.55GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 0.44GiB; the rest of the memory reserved for KV Cache is 5.84GiB.
INFO 01-24 15:12:07 executor_base.py:107] # CUDA blocks: 748, # CPU blocks: 512
INFO 01-24 15:12:07 executor_base.py:112] Maximum concurrency for 2048 tokens per request: 5.84x
INFO 01-24 15:12:09 model_runner.py:1448] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as ne

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:10<00:00,  3.25it/s]

INFO 01-24 15:12:19 model_runner.py:1574] Graph capturing finished in 11 secs, took 0.24 GiB
INFO 01-24 15:12:19 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 12.90 seconds


In [11]:
from vllm import SamplingParams
import torch
input_ids = [1] * 317
sampling_params = SamplingParams(temperature=0.0,
                                     max_tokens=300,
                                     stop_token_ids= [2])
# outputs = language_model.generate(prompt_token_ids=input_ids, sampling_params=sampling_params)
multimodal_embeddings = torch.load("logs/multimodal_embeddings_cpu.pt", map_location="cpu", weights_only=True).squeeze(0)

In [18]:
tokenizer = AutoTokenizer.from_pretrained("logs/llama-bridge")
tokenizer("<PAD>")

{'input_ids': [1, 32000], 'attention_mask': [1, 1]}

In [13]:
outputs  = language_model.generate({"prompt_token_ids": input_ids, "multi_modal_data": {"image": multimodal_embeddings}}, sampling_params=sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.58s/it, est. speed input: 69.19 toks/s, output: 62.64 toks/s]


In [14]:
for o in outputs:
    generated_text = o.outputs[0].token_ids
    print(generated_text)
    print(len(generated_text))

(15484, 278, 16699, 837, 295, 265, 373, 278, 304, 20466, 29889, 16507, 2190, 29901, 25249, 278, 5075, 304, 278, 1492, 322, 701, 29892, 322, 2058, 278, 16699, 837, 295, 265, 373, 278, 304, 20466, 29889, 478, 3235, 8979, 1307, 438, 29933, 17637, 29903, 29901, 263, 13328, 805, 6150, 518, 29906, 29906, 29892, 29871, 29896, 29906, 29929, 29892, 29871, 29947, 29900, 29892, 29871, 29896, 29945, 29906, 1402, 278, 9088, 518, 29896, 29896, 29892, 29871, 29896, 29892, 29871, 29906, 29946, 29896, 29892, 29871, 29906, 29945, 29900, 1402, 263, 304, 29891, 330, 3055, 17615, 518, 29896, 29896, 29900, 29892, 29871, 29896, 29945, 29945, 29892, 29871, 29896, 29953, 29900, 29892, 29871, 29906, 29906, 29929, 1402, 263, 7254, 13950, 518, 29896, 29945, 29945, 29892, 29871, 29929, 29953, 29892, 29871, 29906, 29906, 29955, 29892, 29871, 29896, 29953, 29946, 29962, 27092, 29911, 3289, 29968, 5195, 29909, 3094, 4214, 29901, 450, 330, 374, 2496, 4225, 304, 4337, 304, 278, 1492, 322, 701, 297, 1797, 304, 2058, 278

In [15]:
o.outputs[0].text

'Place the watermelon on the towel. PLAN: Move the arm to the right and up, and place the watermelon on the towel. VISIBLE OBJECTS: a yellow spoon [22, 129, 80, 152], the scene [11, 1, 241, 250], a toy giraffe [110, 155, 160, 229], a blue cloth [155, 96, 227, 164] SUBTASK REASONING: The gripper needs to move to the right and up in order to place the watermelon on the towel. SUBTASK: Move the arm to the right and up. MOVE REASONING: The gripper needs to move to the right and up in order to place the watermelon on the towel. MOVE: Move right up. GRIPPER POSITION: [119, 90, 138, 84, 154, 80, 169, 80, 183, 85] ACTION: 塔ფ斯만助昭Ÿ'